In [1]:
import torch
import syft as sy
import pandas as pd
from syft.lib.python.list import List

### Make the VM and its client

In [2]:
server = sy.VirtualMachine(name="server")
client = server.get_root_client()
client2 = server.get_client()

In [3]:
print(type(server))
print(server)
print(type(client))
print(client)
print(client2)

<class 'syft.core.node.vm.vm.VirtualMachine'>
VirtualMachine: server: <UID: edd8baafc0a54046a27a67cdce75cf7b>
<class 'syft.core.node.vm.client.VirtualMachineClient'>
<VirtualMachineClient: server Client>
<VirtualMachineClient: server Client>


### Check out the remote modules

In [4]:
remote_pd = client.pandas
remote_torch = client.torch
remote_np = client.numpy
remote_vision = client.torchvision

In [5]:
print(type(remote_pd))
remote_pd

<class 'syft.ast.module.Module'>


Module:
	.DataFrame -> <syft.ast.klass.Class object at 0x7f92d614b6a0>
	.Series -> <syft.ast.klass.Class object at 0x7f92d614b700>
	.CategoricalDtype -> <syft.ast.klass.Class object at 0x7f92d614b760>
	.Categorical -> <syft.ast.klass.Class object at 0x7f92d614b7c0>
	.json_normalize -> <syft.ast.callable.Callable object at 0x7f92d614b880>

In [6]:
remote_np

Module:
	.ndarray -> <syft.ast.klass.Class object at 0x7f249ef6e520>

In [7]:
remote_vision

Module:
	.__version__ -> <syft.ast.static_attr.StaticAttribute object at 0x7f249f18dd60>
	.transforms -> Module:
		.Compose -> <syft.ast.klass.Class object at 0x7f249f18de20>
		.ToTensor -> <syft.ast.klass.Class object at 0x7f249f18de80>
		.Normalize -> <syft.ast.klass.Class object at 0x7f249f18df40>
		.CenterCrop -> <syft.ast.klass.Class object at 0x7f249f049340>
		.ColorJitter -> <syft.ast.klass.Class object at 0x7f249f049400>
		.FiveCrop -> <syft.ast.klass.Class object at 0x7f249f049460>
		.Grayscale -> <syft.ast.klass.Class object at 0x7f249f049520>
		.Pad -> <syft.ast.klass.Class object at 0x7f249f049580>
		.RandomAffine -> <syft.ast.klass.Class object at 0x7f249f049640>
		.RandomApply -> <syft.ast.klass.Class object at 0x7f249f049700>
		.RandomCrop -> <syft.ast.klass.Class object at 0x7f249f0497c0>
		.RandomGrayscale -> <syft.ast.klass.Class object at 0x7f249f049880>
		.RandomHorizontalFlip -> <syft.ast.klass.Class object at 0x7f249f049940>
		.RandomPerspective -> <syft.ast.klass

In [8]:
remote_torch

Module:
	.Tensor -> <syft.ast.klass.Class object at 0x7f249f31e820>
	.BFloat16Tensor -> <syft.ast.klass.Class object at 0x7f249f31e880>
	.BoolTensor -> <syft.ast.klass.Class object at 0x7f249f31e8e0>
	.ByteTensor -> <syft.ast.klass.Class object at 0x7f249f31e940>
	.CharTensor -> <syft.ast.klass.Class object at 0x7f249f31e9a0>
	.DoubleTensor -> <syft.ast.klass.Class object at 0x7f249f31ea00>
	.FloatTensor -> <syft.ast.klass.Class object at 0x7f249f31ea60>
	.HalfTensor -> <syft.ast.klass.Class object at 0x7f249f31eac0>
	.IntTensor -> <syft.ast.klass.Class object at 0x7f249f31eb20>
	.LongTensor -> <syft.ast.klass.Class object at 0x7f249f31eb80>
	.ShortTensor -> <syft.ast.klass.Class object at 0x7f249f31ebe0>
	.nn -> Module:
		.Parameter -> <syft.ast.klass.Class object at 0x7f249f31ed00>
		.Module -> <syft.ast.klass.Class object at 0x7f249f26e1c0>
		.Conv2d -> <syft.ast.klass.Class object at 0x7f249f26e6a0>
		.Dropout2d -> <syft.ast.klass.Class object at 0x7f249f26eac0>
		.Linear -> <syft.

### Sending server's data to the client

In [6]:
x = torch.tensor([1,2,3]).tag("hey", "there", "buddy")
x.description = "wow what a day"
x_ptr = x.send(client)

In [7]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: fe84a322da834202b9be221e0739ed2d>,"[hey, there, buddy]",wow what a day,<class 'torch.Tensor'>


In [8]:
client2.store.pandas

,ID,Tags,Description,object_type
0,<UID: fe84a322da834202b9be221e0739ed2d>,"[hey, there, buddy]",wow what a day,<class 'torch.Tensor'>


### Sending client's data to the server

In [9]:
server = sy.VirtualMachine(name="server")
client = server.get_root_client()
client_data = torch.randn(size=(1, 100))
client_data_ptr = client_data.send(server)

AttributeError: 'VirtualMachine' object has no attribute 'send_immediate_msg_without_reply'

### Checking out client's device (cuda or CPU)

In [9]:
client_cuda_ptr = remote_torch.cuda.is_available()
client_cuda = client_cuda_ptr.get_copy()
print(f'client has CUDA? {client_cuda}')

client_device_ptr = remote_torch.device("cuda" if client_cuda else "cpu")
print(f'client device: {client_device_ptr.type.get_copy()}')

client has CUDA? True
client device: cuda


In [10]:
t1 = torch.tensor([1, 2])
t2 = torch.tensor([1, 3])
syft_list = List([t1, t2])
syft_list_ptr = syft_list.send(client)

In [11]:
res = syft_list_ptr.get_copy()
res

[tensor([1, 2]), tensor([1, 3])]

In [12]:
tensor_type='uint8'
op_name = '__add__'


In [13]:
x = torch.tensor([-1, 0, 1, 2, 3, 4])
xp = x.send(client)
op_method = getattr(xp, op_name, None)
target_op_method = getattr(x, op_name)

print(op_method)
print(target_op_method)

<bound method get_run_class_method.<locals>.run_class_method of <syft.proxy.torch.TensorPointer object at 0x7f92d2bad640>>
<built-in method __add__ of Tensor object at 0x7f92d2b40700>


In [14]:
result = op_method(xp)  # op(xp, xp)
target_result = target_op_method(x)

local_result = result.get()

assert (local_result == target_result).all()

In [15]:
local_result

tensor([-2,  0,  2,  4,  6,  8])